In [ ]:
import math
import numpy as np
import cv2
import mediapipe as mp

# Create a Mediapipe Holistic object
mp_holistic = mp.solutions.holistic.Holistic()

# Define the temporal window size
W = 10

# Initialize lists to store trajectory lengths and convex hull perimeters
trajectory_lengths_left = []
trajectory_lengths_right = []
convex_hull_perimeters_left = []
convex_hull_perimeters_right = []

# Function to calculate the length of a trajectory
def calculate_trajectory_length(landmark_points):
    length = 0
    for i in range(1, len(landmark_points)):
        dx = landmark_points[i][0] - landmark_points[i-1][0]
        dy = landmark_points[i][1] - landmark_points[i-1][1]
        length += math.sqrt(dx**2 + dy**2)
    return length

# Function to calculate the perimeter of the convex hull
def calculate_convex_hull_perimeter(landmark_points):
    hull = cv2.convexHull(np.array(landmark_points, dtype=np.float32))
    perimeter = cv2.arcLength(hull, closed=True)
    return perimeter


# Function to calculate the geometric entropy
def calculate_geometric_entropy(trajectory_lengths, convex_hull_perimeters):
    entropies = []
    for i in range(len(trajectory_lengths)):
        hk_ti = math.log(2 * trajectory_lengths[i] / convex_hull_perimeters[i])
        entropies.append(hk_ti)
    return entropies

# Initialize video capture from the webcam
video_capture = cv2.VideoCapture(0)

# Main loop for processing frames
while True:
    # Read frame from video source
    ret, frame = video_capture.read()

    # Convert the BGR image to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with Mediapipe Holistic
    results = mp_holistic.process(frame_rgb)

    # Check if landmarks are available
    if results.left_hand_landmarks and results.right_hand_landmarks:
        # Extract landmark points for the left and right hands
        left_hand_landmarks = [(landmark.x, landmark.y) for landmark in results.left_hand_landmarks.landmark]
        right_hand_landmarks = [(landmark.x, landmark.y) for landmark in results.right_hand_landmarks.landmark]

        # Calculate trajectory lengths for the left and right hands
        left_trajectory_length = calculate_trajectory_length(left_hand_landmarks)
        right_trajectory_length = calculate_trajectory_length(right_hand_landmarks)

        # Calculate convex hull perimeters for the left and right hands
        left_convex_hull_perimeter = calculate_convex_hull_perimeter(left_hand_landmarks)
        right_convex_hull_perimeter = calculate_convex_hull_perimeter(right_hand_landmarks)

        # Add the trajectory lengths and convex hull perimeters to the respective lists
        trajectory_lengths_left.append(left_trajectory_length)
        trajectory_lengths_right.append(right_trajectory_length)
        convex_hull_perimeters_left.append(left_convex_hull_perimeter)
        convex_hull_perimeters_right.append(right_convex_hull_perimeter)

        # Ensure the lists do not exceed the temporal window size
        if len(trajectory_lengths_left) > W:
            trajectory_lengths_left.pop(0)
        if len(trajectory_lengths_right) > W:
            trajectory_lengths_right.pop(0)
        if len(convex_hull_perimeters_left) > W:
            convex_hull_perimeters_left.pop(0)
        if len(convex_hull_perimeters_right) > W:
            convex_hull_perimeters_right.pop(0)

        # Calculate geometric entropy for the left and right hands
        entropy_left = calculate_geometric_entropy(trajectory_lengths_left, convex_hull_perimeters_left)
        entropy_right = calculate_geometric_entropy(trajectory_lengths_right, convex_hull_perimeters_right)

        # Calculate symmetry score between 0 and 100
        symmetry_scores = []
        for i in range(len(entropy_left)):
            symmetry_score = 100 * entropy_left[i] / entropy_right[i]
            symmetry_scores.append(symmetry_score)
            print(symmetry_score)

        # Analyze the symmetry scores for further processing or visualization

    # Display the frame with landmarks
    cv2.imshow('Frame', frame)

    # Check for key press to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video source and close windows
video_capture.release()
cv2.destroyAllWindows()
